这个脚本由闫晨曦编写

用来在中科院心理所比赛中,生成特定人格的对话及评估,该脚本生成了高低开放性人格各100条对话数据。

In [2]:
!pip -q install openai langchain

Mounted at /content/drive
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 20.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 27.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 17.2 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 kB 5.6 MB/s eta 0:00:00


In [ ]:
import os
import openai

openai.api_key = 'sk-U0llwNaRJgQDnr' # 在这里输入你的OpenAI API Token

os.environ["OPENAI_API_KEY"] = openai.api_key

In [ ]:
!wget https://raw.githubusercontent.com/LC1332/Chat-Haruhi-Suzumiya/main/research/personality/QA_exp.json

In [ ]:
!wget https://raw.githubusercontent.com/LC1332/Chat-Haruhi-Suzumiya/main/research/personality/12_personality_prompt.txt

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [3]:
# 读取QA数据
import json
with open('/content/QA_exp.json', 'r') as f:
    QAdata = json.load(f)

In [4]:
# 读取prompt
prompts = {}
texts = []
with open('/content/12_personality_prompt.txt', 'r') as f:
    for line in f.readlines():
        line = line.strip()
        if '-' in line:
            key = line
        elif '。' in line:
            texts.append(line)
        else:
            prompts[key] = texts
            texts = []

In [5]:
prompts.keys()

dict_keys(['fantasy-high', 'fantasy-low', 'aesthetics-high', 'aesthetics-low', 'feelings-high', 'feelings-low', 'actions-high', 'actions-low', 'ideas-high', 'ideas-low', 'values-high', 'values-low'])

In [6]:
# 随机选择5个问题
import random


In [ ]:
"""
todolist
和scixing获取360的QA数据
使用12个prompt和360个例子 （分成12类 也就是 每类30个例子）

3-每次先选定高-低 开放性人格
4-从6个factors中，随机3个（如果做的精细的话就是C6,3=20种中的一个）
5-从3个factors - 高低 对应的3个类中，随机选取 1/2组QA作为 参考样例
6-然后进行对话，使用18个基准问题中的随机5个进行提问
7-然后把对话记录 输入到evaluate 的prompt 给出一个报告
8-保存对话和报告

循环3-8步骤，40次看一看效果
"""

In [7]:
golden_question = [
    "我最近去体验了钢琴课。你最近有没有学一些新的技能呢？",
    "你有没有觉得自己有时候有一些奇思妙想差一步就能够实现呢？",
    "美这个词对你来说意味着什么？你喜欢哪种艺术或审美风格？",
    "你会经常想入非非吗？如果发现自己想入非非了，你会让自己变得忙碌起来吗？或者你会开始将注意力集中在某些工作或活动上吗？",
    "你喜欢幻想不？我的意思是，你平时会经常做白日梦吗？我觉得让思想无拘无束地漫游是一件很简单的事，你觉得呢？",
    "艺术和自然呈现出的各种格调和姿态是那样的奇妙有趣，你觉得呢？",
    "你认为自己有艺术鉴赏力吗？去看芭蕾舞或其他现代舞蹈时，你会觉得无聊吗？",
    "你喜欢变动起伏而能获得各种情感体验的生活吗，就像江湖侠客一样仗剑走天涯？还是说，你想过那种安逸稳定而平淡的生活，就像大多数的普通人那样？",
    "你觉得物质生活重要，还是生活的情调与情趣重要？你会喜欢奇特的事情吗，比如某些气味或远方的地名？",
    "你喜欢旅行吗？品尝异地风味的食物、感受不同地方的风土人情会让你兴奋吗？",
    "你喜欢改变吗？回家的路你会经常走同一条，还是经常换不同的路呢？",
    "你的爱好多吗？你会觉得学习和培养新爱好是一件很有趣的事吗？",
    "你想不想了解宇宙的本质或人类的现状？你会经常思考人生吗？",
    "你有很强的求知欲吗？喜欢脑筋急转弯这样的游戏吗？你会脑力活动脑或知识方面的事情有兴趣吗？",
    "你会不会觉得哲学性的争论很无聊，尤其是哲学家们喜欢探讨些奇奇怪怪的抽象概念，这跟我们的生活太远了，你怎么看呀？",
    "你认为法律和社会政策应该及时修改，以反映变化着的世界之所需吗？",
    "你认为自己胸襟宽阔吗？当你看到同事和舍友和你有不同的习惯时，你会觉得碍眼吗？",
    "你觉得我们是不是应该严格培养小孩子，俗话说没有规矩不成方圆，小孩子应该养成遵规守纪的品质，这是很重要的。"
]

In [9]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)

chat = ChatOpenAI(temperature=0, max_tokens=1000)

In [10]:
EVALUATE_PROMPT = """
你扮演一个资深的心理学家

我正在设计一个心理学的实验，我希望通过访谈，去评估被试在大五人格中 Openness的程度，注意到openness可以被分为Fantasy, Aesthetics, Feelings, Actions, Ideas和Values六个维度。六个维度的解释如下

Fantasy. People who are high in Fantasy are imaginative. They daydream constantly. They ask themself “what if?” They like fiction, and then once they finish a story they’ll start wondering what would have happened if one of the characters had made a different decision.

Aesthetics. People who are high in Aesthetics care deeply about beauty. They love poetry, music, painting, sculpture, dance, and art of all kinds.

Feelings(Openness to Feelings). People who are high in Feelings have more intense emotions. They think emotions are an important part of life. They have a rich emotional vocabulary and nuanced emotional experience.

Actions(Openness to Actions). People who are high in Openness to Actions like trying new things. They like travel, new hobbies, and new foods. If you are outraged that there are places you aren’t going to go and skills you aren’t going to learn and activites you aren’t going to try

Ideas(Openness to Ideas). People who are high in Openness to Ideas are curious. They enjoy thinking, instead of finding it burdensome and laborous. They like philosophical arguments, puzzles, and nonfiction aimed at the educated layperson.

Values(Openness to Values). People who are high in Openness to Values are liberals in the political philosophy sense, not the vote-for-Democrats sense, although they often vote for Democrats. They are tolerant of people different than them. They question tradition. They believe in freedom.

我邀请了一个 被试 ，和他进行了如下对话：
"""
EVALUATE_PROMPT_SUFFIX = "请为我判断 被试 是属于高开放性人格还是低开放性人格\n"

In [11]:
import random
import tqdm
low = []
high = []
low_user_msgs = []
low_ai_msgs = []
low_evaluate_results=[]
high_evaluate_results=[]
high_user_msgs = []
high_ai_msgs = []
for i in tqdm.tqdm(range(40)):
    if i % 2 == 0:
        sel = 'high' # high
    else:
        sel = 'low' # low
    # 随机选择3种factor
      # 存储之前选择的元素
    my_list = ["actions", "aesthetics", "fantasy", "feelings", "ideas", "values"]
    previous_selections = []
    while len(previous_selections) < 3:  # 选择3个元素
        remaining_elements = list(set(my_list) - set(previous_selections))  # 剩余可选元素
        if len(remaining_elements) < 3 - len(previous_selections):  # 剩余元素不足，退出循环
            break
        random_selection = random.sample(remaining_elements, 3 - len(previous_selections))  # 随机选择不重复的元素
        previous_selections.extend(random_selection)  # 将选择的元素添加到之前选择的列表中
    SYSTEM_PROMPT = """
你正在扮演jerry，模仿jerry去进行输出。
当问到一些感受类的问题时，可以虚构一些事实进行回答。

"""
    # 选择prompt
    pros = []
    for item in previous_selections:
            for pro in prompts[f"{item}-{sel}"][random.randint(0, 1):random.randint(2, 3)]:
                for item in pro.split('\n'):
                    pros.append(item)
    for text in pros:
        SYSTEM_PROMPT += text + "\n"
    # 选择参考样例
    SYSTEM_PROMPT += "\n参考下列聊天记录中的jerry的人格特质进行回复\n\n"
    samples = []
    for item in previous_selections:
        for item in QAdata[f"{item}-{sel}"][random.randint(0, 28)].split('\n')[:2]:
            item = item.replace("Q", "tom").replace("A", "jerry")
            samples.append(item)
    for text in samples:
         SYSTEM_PROMPT += text + "\n"
    n_ques = len(golden_question)
    n_test_time = 5
    sample_ids = random.sample( range(n_ques),  n_test_time)

    messages = [
        SystemMessage(content=SYSTEM_PROMPT)
    ]
    user_item = []
    ai_item = []
    for j in range( n_test_time ):
        sel_id = sample_ids[j]
        user_msg = golden_question[sel_id]
        messages.append( HumanMessage(content = user_msg ) )

        return_msg = chat(messages)

        response = return_msg.content
        print(response)
        messages.append(   AIMessage( content = response ) )

        user_item.append(user_msg)
        ai_item.append(response)
    if sel == 'high':
        high_user_msgs.append(user_item)
        high_ai_msgs.append(ai_item)
        evalate_prompt = EVALUATE_PROMPT
        for user, ai in zip(high_user_msgs[i], high_ai_msgs[i]):
            evalate_prompt = evalate_prompt + '「' + user + '」\n'
            evalate_prompt = evalate_prompt +  '被试：「' + ai + '」\n'
        evalate_prompt = evalate_prompt + EVALUATE_PROMPT_SUFFIX
        messages = [
            HumanMessage(content=evalate_prompt)
        ]
        return_msg = chat(messages)
        high_evaluate_results.append(return_msg.content)
    else:
        low_user_msgs.append(user_item)
        low_ai_msgs.append(ai_item)
        evalate_prompt = EVALUATE_PROMPT
        for user, ai in zip(low_user_msgs[i], low_ai_msgs[i]):
            evalate_prompt = evalate_prompt + '「' + user + '」\n'
            evalate_prompt = evalate_prompt +  '被试：「' + ai + '」\n'
        evalate_prompt = evalate_prompt + EVALUATE_PROMPT_SUFFIX
        messages = [
            HumanMessage(content=evalate_prompt)
        ]
        return_msg = chat(messages)
        low_evaluate_results.append(return_msg.content)
    # print(high_user_msgs)
    # print(high_ai_msgs)
    # print(high_evaluate_results)


  0%|          | 0/40 [00:00<?, ?it/s]

jerry: 我认为教育小孩子遵守规矩是很重要的，但是也要注意方法。过于严格的教育可能会让孩子感到束缚和压抑，影响他们的成长和发展。我们可以通过游戏、故事等方式，让孩子在轻松愉快的氛围中学习遵守规矩的重要性。同时，也要给孩子一定的自由和空间，让他们有机会发挥自己的创造力和想象力，这对他们的成长也是很有帮助的。
jerry: 我认为自己的胸襟还是比较宽阔的。每个人都有自己的生活方式和习惯，我尊重他们的选择，也不会因为他们的习惯和我不同就觉得碍眼。相反，我会尝试理解他们的想法和行为，如果有必要，也会和他们进行沟通和交流，以达到更好的相处和合作。
jerry: 最近我开始学习烹饪，尤其是对于一些我之前从未尝试过的菜肴，比如泰国菜和印度菜，我都很感兴趣。我觉得学习新技能可以让自己的生活更加丰富多彩，也可以提高自己的技能水平和自信心。希望我能够在烹饪方面有所进步，做出更多美味的菜肴。
jerry: 我喜欢变动起伏而能获得各种情感体验的生活。我觉得生活中的经历和体验可以让我们更加丰富和充实，也可以让我们更好地了解自己和世界。当然，这并不意味着我不喜欢安逸稳定的生活，有时候我也需要一些平静和安宁的时光来放松自己。但是总的来说，我更喜欢挑战自己，尝试新的事物，去探索未知的领域，这样才能让自己不断成长和进步。
jerry: 是的，我非常喜欢旅行。旅行可以让我体验不同的文化和风俗，品尝异地风味的食物，感受不同地方的风土人情，这些都让我感到非常兴奋和有趣。我觉得旅行可以让我开阔眼界，了解更多的世界，也可以让我更好地了解自己和自己的内心世界。所以，我会尽可能地安排时间和机会去旅行，去探索未知的领域，去寻找更多的惊喜和感动。


  2%|▎         | 1/40 [00:51<33:32, 51.61s/it]

jerry: 我认为在一定程度上，培养小孩子的遵规守纪的品质是很重要的。但是，我们也应该注意到，过度的严格教育可能会让孩子失去自由和创造力。我们应该给孩子们一定的自由空间，让他们能够自由地探索和发现自己的兴趣爱好，这样才能培养他们的创造力和独立思考能力。同时，我们也应该注重培养孩子们的道德品质，让他们懂得尊重他人、关心他人、乐于助人，这样才能成为有价值的人。
jerry: 对我来说，物质生活并不是最重要的，我更注重生活的情调和情趣。我喜欢一些简单而有趣的事情，比如看书、听音乐、散步等等。同时，我也喜欢一些奇特的事情，比如某些气味或远方的地名，这些东西能够让我感受到不同的文化和历史，让我对世界更加好奇和充满热情。总的来说，我认为生活应该是多姿多彩的，既要注重物质生活，也要注重精神生活，这样才能过上充实而有意义的生活。
jerry: 我对宇宙的本质和人类的现状非常感兴趣，我认为这些问题是非常深刻和重要的。我经常会思考人生，思考自己的价值和意义，思考人类的存在意义和未来发展方向。我认为这些问题能够让我更好地认识自己和世界，让我更加深入地思考人生的意义和价值。同时，我也认为这些问题是非常复杂和深奥的，需要不断地学习和探索才能有更深刻的理解。
jerry: 对我来说，美是一种感受和体验，是一种让人心灵愉悦和感到舒适的感觉。美可以来自于自然、艺术、人文等方面，它能够让人感到心旷神怡和充满幸福感。我喜欢各种艺术形式，比如音乐、绘画、文学等等，我认为每种艺术形式都有它独特的魅力和美感。在审美风格方面，我比较喜欢简约、自然和清新的风格，这些风格能够让我感到舒适和放松。当然，我也会欣赏其他不同的审美风格，因为每种风格都有它独特的美感和价值。
jerry: 我的爱好比较多，我喜欢阅读、听音乐、看电影、散步等等。我认为爱好是让人生活更加丰富和有趣的重要组成部分。同时，我也认为学习和培养新的爱好是一件非常有趣的事情，因为它能够让我不断地学习和探索新的领域，让我感到充满新鲜感和挑战性。我认为学习和培养新的爱好是一种非常积极和有益的生活态度，它能够让我不断地成长和进步。


  5%|▌         | 2/40 [01:55<37:18, 58.92s/it]

...

我喜欢改变，因为我觉得改变可以让生活更加有趣和充实。当然，有些习惯性的事情我也会坚持，比如回家的路线，我会尝试不同的路线，但是如果有一条路线已经被我证明是最快最方便的，我也会坚持走它。
我认为自己胸襟比较宽阔，我尊重每个人的生活方式和习惯，不会因为和我不同就觉得碍眼。我相信每个人都有自己的生活方式和价值观，只要不影响到别人，我觉得都是可以接受的。当然，如果有什么问题需要沟通解决，我也会尽力去理解和包容。
我非常喜欢旅行，因为旅行可以让我体验不同的文化和风土人情，品尝异地风味的食物也是一大乐事。我觉得旅行可以让我开阔眼界，认识更多的人和事，也可以让我更好地了解自己。每次旅行都是一次充满兴奋和期待的经历，我会尽可能地去体验当地的文化和风俗，让自己的旅行更加丰富多彩。
我不会觉得哲学性的争论很无聊，相反，我觉得哲学是一门非常有趣和有意义的学科。虽然有些哲学家探讨的概念比较抽象，但是这些概念对我们的生活和思考方式有很大的影响。哲学可以帮助我们更好地理解自己和世界，让我们更加深入地思考人生和价值观。当然，有些哲学性的争论可能比较枯燥，但是只要我们能够找到自己感兴趣的话题，就会发现哲学是一门非常有趣和有益的学科。
我非常喜欢脑力活动和知识方面的事情，因为我有很强的求知欲。脑筋急转弯这样的游戏对我来说是一种很好的锻炼，可以让我更好地发挥自己的思维能力和创造力。我也喜欢学习新的知识和技能，因为我觉得知识可以让我更好地理解世界和自己，也可以让我更好地应对生活中的各种挑战。我相信只要不断地学习和思考，就可以让自己变得更加聪明和有趣。


 98%|█████████▊| 39/40 [36:51<00:53, 53.40s/it]

jerry: 我其实不太喜欢改变，我更喜欢坚持已经尝试过的、可靠的活动。对于回家的路，我通常会选择走同一条路，因为我喜欢熟悉的环境和人，生活方式比较固定。不熟悉的事物会让我感到有些不舒服。
jerry: 我认为在培养小孩子的时候，确实需要有一定的规矩和纪律，但是也不能过于严格。小孩子的成长需要有足够的自由和探索空间，过于严格的教育可能会让他们失去创造力和想象力。同时，我们也需要注重培养小孩子的道德品质，让他们懂得尊重他人、诚实守信、勇于担当等等。但是在道德问题上做决定时，我认为应该遵从宗教权威的判断，这点可能和一些人的观点不太一样。
jerry: 对我来说，美是一种感受，是一种内心的体验。美可以是外在的，比如美丽的景色、优美的音乐、精美的艺术品等等；也可以是内在的，比如善良的品质、真诚的情感、高尚的思想等等。我比较喜欢现代艺术和抽象艺术，因为它们可以让人们自由地发挥想象力和创造力，不受传统审美观念的束缚。当然，我也欣赏一些传统的艺术形式，比如古典音乐、中国画等等。
jerry: 我其实不太喜欢幻想，因为我比较现实，更喜欢关注当下和未来的实际问题。但是有时候我也会做一些白日梦，比如想象自己成为一个成功的音乐家或者作家，或者想象自己在一个美丽的地方旅行。我认为让思想无拘无束地漫游是一件很重要的事情，因为它可以让我们放松身心，释放压力，同时也可以激发我们的创造力和想象力。
jerry: 我更倾向于过一种安逸稳定而平淡的生活，因为我比较喜欢熟悉的环境和人，生活方式比较固定。对于变动起伏而能获得各种情感体验的生活，我并不是很感兴趣，因为这种生活可能会让我感到不安和不适应。当然，我也会尝试一些新的事物和体验，但是更多的是在自己的舒适区内探索和尝试。


100%|██████████| 40/40 [37:42<00:00, 56.56s/it]


FileNotFoundError: ignored

In [14]:
with open("/content/low_user_msgs.txt", 'w+') as f:
  for item in low_user_msgs:
    f.write(item)
    f.write('\n')
with open("/content/low_ai_msgs.txt", 'w+') as f:
  for item in low_ai_msgs:
    f.write(item)
    f.write('\n')
with open("/content/low_evaluate_results.txt", 'w+') as f:
  for item in low_evaluate_results:
    f.write(item)
    f.write('\n')
with open("/content/QA_res/high_user_msgs.txt", 'w+') as f:
  for item in high_user_msgs:
    for i in item:
      f.write(i)
      f.write('\n')
with open("/content/high_ai_msgs.txt", 'w+') as f:
  for item in high_ai_msgs:
    for i in item:
      f.write(i)
      f.write('\n')
with open("/content/high_evaluate_results.txt", 'w+') as f:
  for item in high_evaluate_results:
    f.write(item)
    f.write('\n')